In [78]:
%load_ext autoreload
%autoreload 2


from src.forecast.cnn import cnnpred_2d, datagen, f1macro, testgen, scale_inputs, precision_m, precision_loss
from src.utils.utils import load, add_variables

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Testowanie CNN 2D

In [79]:
CURR = "EURUSD"
INTERVAL = "4h"

In [80]:
# load data
eurusd = load(currency=CURR, interval=INTERVAL)
eurusd = add_variables(ohlc=eurusd)

# clean data
eurusd.drop(columns=['target_value'], inplace=True)
eurusd = eurusd[['Close', 'target_direction']]

# scale inputs
eurusd = scale_inputs(eurusd, targetcol=["target_direction"]) # use different scaler to preserver more info between small values

### Training model

In [42]:
checkpoint_path = "./cp2d-{epoch}-{val_f1macro:.2f}.h5"
callbacks = [
    ModelCheckpoint(checkpoint_path,
                    monitor='val_f1macro', mode="max", verbose=0,
                    save_best_only=True, save_weights_only=False, save_freq="epoch")
]

In [47]:
import tensorflow as tf
from keras.layers import Input, Conv1D, Flatten, Dense, Dropout
from keras.models import Sequential

In [48]:
seq_len = 10
batch_size = 128
n_epochs = 5
n_features = 1

class_weight={
        0: 1,
        1: 1
}


def cnnpred_2d(seq_len, n_features):
    model = Sequential(
        [
            Input(shape=(seq_len, n_features, 1)),
            Conv1D(10, kernel_size=16, activation='sigmoid', padding='same'),
            
            Flatten(),
            Dense(256, activation='sigmoid'),
            Dense(128, activation='sigmoid'),
            Dropout(0.1),
            Dense(2, activation="softmax"),
        ]
    )
    return model

 
model = cnnpred_2d(seq_len=seq_len, n_features=n_features)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=["acc", f1macro, precision_m])


In [77]:

model.fit(datagen(df=eurusd,
                  seq_len=seq_len,
                  batch_size=batch_size,
                  targetcol=["target_direction"],
                  kind="train"),
          validation_data=datagen(df=eurusd,
                                  seq_len=seq_len,
                                  batch_size=batch_size,
                                  targetcol=["target_direction"],
                                  kind="valid"),
          epochs=n_epochs, 
          steps_per_epoch=200, 
          validation_steps=10,
          class_weight=class_weight,
          verbose=1,
          callbacks=callbacks)

Epoch 1/5


2023-03-29 23:42:36.078717: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - ETA: 0s - loss: 0.7622 - acc: 0.5018 - f1macro: 0.6541 - precision_m: 0.3613

2023-03-29 23:42:49.973081: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 16s 75ms/step - loss: 0.7622 - acc: 0.5018 - f1macro: 0.6541 - precision_m: 0.3613 - val_loss: 0.6628 - val_acc: 1.0000 - val_f1macro: 0.6550 - val_precision_m: 0.3672
Epoch 2/5
200/200 [==============================] - 15s 73ms/step - loss: 0.6781 - acc: 0.5025 - f1macro: 0.6541 - precision_m: 0.3609 - val_loss: 0.6879 - val_acc: 0.0000e+00 - val_f1macro: 0.6552 - val_precision_m: 0.3695
Epoch 3/5
200/200 [==============================] - 14s 72ms/step - loss: 0.6849 - acc: 0.5063 - f1macro: 0.6536 - precision_m: 0.3588 - val_loss: 0.6832 - val_acc: 1.0000 - val_f1macro: 0.6544 - val_precision_m: 0.3625
Epoch 4/5
200/200 [==============================] - 14s 72ms/step - loss: 0.6928 - acc: 0.4808 - f1macro: 0.6545 - precision_m: 0.3638 - val_loss: 0.7012 - val_acc: 1.0000 - val_f1macro: 0.6561 - val_precision_m: 0.3688
Epoch 5/5
200/200 [==============================] - 14s 72ms/step - loss: 0.6912 - acc: 0.4861 - f1macro: 0.6545 - precis

In [78]:
# Prepare test data
test_data, test_target = testgen(eurusd, seq_len, ["target_direction"])

# Test the model
test_out = model.predict(test_data)
test_pred = (test_out > 0.1).astype(int)
print(f"accuracy: {accuracy_score(test_pred, test_target)}")
print(f"MAE: {mean_absolute_error(test_pred, test_target)}")
print(f"F1: {f1_score(test_pred, test_target)}")

 66/330 [=====>........................] - ETA: 0s

2023-03-29 23:43:52.349919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


330/330 [==============================] - 1s 2ms/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets

In [ ]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_27 (Conv1D)          (None, 10, 1, 10)         170       
                                                                 
 flatten_13 (Flatten)        (None, 100)               0         
                                                                 
 dense_18 (Dense)            (None, 256)               25856     
                                                                 
 dense_19 (Dense)            (None, 128)               32896     
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_20 (Dense)            (None, 1)                 129       
                                                                 
Total params: 59,051
Trainable params: 59,051
Non-tra

In [79]:
test_out

array([[0.5008456 , 0.49915445],
       [0.5008456 , 0.49915445],
       [0.5008456 , 0.49915445],
       ...,
       [0.5008456 , 0.49915445],
       [0.5008456 , 0.49915445],
       [0.5008456 , 0.49915445]], dtype=float32)

In [22]:
eurusd['target_direction'].value_counts()/eurusd.shape[0]

0    0.640443
1    0.359557
Name: target_direction, dtype: float64

In [23]:
pd.DataFrame(test_pred).value_counts()

1    10556
dtype: int64

# TESTING

In [117]:
x = tf.constant([1., 2., 3., 4., 5.])
x

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([1., 2., 3., 4., 5.], dtype=float32)>

In [118]:
x = eurusd['target_direction'].values[:20]

In [119]:
x = np.reshape(x, [1, 20, 1]).astype(float)

In [120]:
x

array([[[0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.]]])

In [126]:
conv = Conv1D(filters=2, kernel_size=2, padding='same')
conv.weights

[]

In [127]:
x_conv = conv(x)
x_conv

<tf.Tensor: shape=(1, 20, 2), dtype=float32, numpy=
array([[[ 0.        ,  0.        ],
        [ 0.36989117, -0.9865818 ],
        [-0.00652289, -0.33389926],
        [-0.37641406,  0.65268254],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.36989117, -0.9865818 ],
        [-0.00652289, -0.33389926],
        [-0.37641406,  0.65268254],
        [ 0.        ,  0.        ],
        [ 0.36989117, -0.9865818 ],
        [-0.37641406,  0.65268254],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.36989117, -0.9865818 ],
        [-0.37641406,  0.65268254],
        [ 0.36989117, -0.9865818 ],
        [-0.37641406,  0.65268254],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]], dtype=float32)>

In [129]:
avg_pool = tf.keras.layers.AveragePooling1D(pool_size=3, strides=1, padding="same")
avg_pool(x)

<tf.Tensor: shape=(1, 20, 1), dtype=float32, numpy=
array([[[0.        ],
        [0.33333334],
        [0.6666667 ],
        [0.6666667 ],
        [0.33333334],
        [0.        ],
        [0.33333334],
        [0.6666667 ],
        [0.6666667 ],
        [0.33333334],
        [0.33333334],
        [0.33333334],
        [0.33333334],
        [0.        ],
        [0.33333334],
        [0.33333334],
        [0.6666667 ],
        [0.33333334],
        [0.33333334],
        [0.        ]]], dtype=float32)>